<a href="https://colab.research.google.com/github/pushp0119/UML-501-ML/blob/main/MLassign4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://books.toscrape.com/catalogue/page-{}.html"
books = []

page = 1
while True:
    url = base_url.format(page)
    r = requests.get(url)
    if r.status_code != 200:
        break
    soup = BeautifulSoup(r.text, "html.parser")
    items = soup.find_all("article", class_="product_pod")
    if not items:
        break

    for item in items:
        title = item.h3.a["title"]
        price = item.find("p", class_="price_color").text
        availability = item.find("p", class_="instock availability").text.strip()
        star = item.p["class"][1]  # e.g. "Three"
        books.append([title, price, availability, star])
    page += 1

df = pd.DataFrame(books, columns=["Title", "Price", "Availability", "Star Rating"])
df.to_csv("books.csv", index=False)
df.head()

,Title,Price,Availability,Star Rating
0,A Light in the Attic,Â£51.77,In stock,Three
1,Tipping the Velvet,Â£53.74,In stock,One
2,Soumission,Â£50.10,In stock,One
3,Sharp Objects,Â£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,Five


Q2

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.imdb.com/chart/top/"
r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(r.text, "html.parser")

movies = []
rows = soup.select("tbody.lister-list tr")

for i, row in enumerate(rows, start=1):
    title = row.select_one(".titleColumn a").text.strip()
    year = row.select_one(".secondaryInfo").text.strip("()")
    rating = row.select_one(".imdbRating strong").text.strip()
    movies.append([i, title, year, rating])

df = pd.DataFrame(movies, columns=["Rank", "Movie Title", "Year", "IMDB Rating"])
df.to_csv("imdb_top250.csv", index=False)
df.head()



,Rank,Movie Title,Year,IMDB Rating


QUESTION3

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.timeanddate.com/weather/"
r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(r.text, "html.parser")

cities = []
# Find ANY table with weather data (look for "°C" or "°F" in it)
tables = soup.find_all("table")
target_table = None
for t in tables:
    if "°C" in t.get_text() or "°F" in t.get_text():
        target_table = t
        break

if target_table:
    for row in target_table.find_all("tr")[1:]:
        cols = row.find_all("td")
        if len(cols) >= 3:
            city = cols[0].get_text(strip=True)
            temp = cols[1].get_text(strip=True)
            cond = cols[2].get_text(strip=True)
            cities.append([city, temp, cond])

df = pd.DataFrame(cities, columns=["City", "Temperature", "Condition"])
df.to_csv("weather.csv", index=False)
df.head()